<a href="https://colab.research.google.com/github/bulut19/mgmt467-analytics-portfolio/blob/main/Lab5_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.


In [3]:
%%bigquery --project boxwood-veld-471119-r6
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-22,mbulut@purdue.edu



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [7]:
%%bigquery --project boxwood-veld-471119-r6
CREATE OR REPLACE TABLE `boxwood-veld-471119-r6.netflix.cleaned_features` AS
WITH wh AS (
  SELECT
    user_id,
    SUM(COALESCE(watch_duration_minutes, 0)) AS total_minutes
  FROM `boxwood-veld-471119-r6.netflix.watch_history`
  GROUP BY user_id
),
rv AS (
  SELECT
    user_id,
    AVG(CAST(rating AS FLOAT64)) AS avg_rating
  FROM `boxwood-veld-471119-r6.netflix.reviews`
  GROUP BY user_id
)
SELECT
  u.user_id,
  u.country,
  u.subscription_plan,
  u.age,
  COALESCE(rv.avg_rating, 0)  AS avg_rating,
  COALESCE(wh.total_minutes, 0) AS total_minutes,
  CASE
    WHEN u.is_active IS NULL THEN NULL               -- keep nulls so we can filter later
    WHEN u.is_active THEN 0 ELSE 1                  -- 0 = active (no churn), 1 = churned
  END AS churn_label
FROM `boxwood-veld-471119-r6.netflix.users` u
LEFT JOIN wh USING (user_id)
LEFT JOIN rv USING (user_id);

Query is running:   0%|          |

""


In [17]:
%%bigquery --project boxwood-veld-471119-r6
CREATE OR REPLACE TABLE `boxwood-veld-471119-r6.netflix.churn_features` AS
SELECT
  user_id, -- Added user_id here
  country           AS region,
  subscription_plan AS plan_tier,
  CASE
    WHEN age IS NULL            THEN 'unknown'
    WHEN age < 18               THEN 'u18'
    WHEN age BETWEEN 18 AND 24  THEN '18-24'
    WHEN age BETWEEN 25 AND 34  THEN '25-34'
    WHEN age BETWEEN 35 AND 44  THEN '35-44'
    WHEN age BETWEEN 45 AND 54  THEN '45-54'
    WHEN age BETWEEN 55 AND 64  THEN '55-64'
    ELSE '65+'
  END                           AS age_band,
  avg_rating,
  total_minutes,
  churn_label
FROM `boxwood-veld-471119-r6.netflix.cleaned_features`
WHERE churn_label IS NOT NULL;

Query is running:   0%|          |

""


In [18]:
%%bigquery --project boxwood-veld-471119-r6
SELECT *
FROM `boxwood-veld-471119-r6.netflix.churn_features`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,region,plan_tier,age_band,avg_rating,total_minutes,churn_label
0,user_02716,Canada,Basic,unknown,0.0,379.6,0
1,user_05854,USA,Basic,18-24,0.0,285.2,0
2,user_03784,Canada,Basic,25-34,0.0,760.3,0
3,user_05931,USA,Basic,25-34,0.0,521.4,0
4,user_07856,USA,Basic,25-34,0.0,255.9,0
5,user_03880,Canada,Basic,35-44,0.0,401.3,0
6,user_07173,Canada,Basic,35-44,0.0,712.9,0
7,user_02729,Canada,Basic,55-64,0.0,977.4,0
8,user_07652,USA,Basic,unknown,0.0,433.0,0
9,user_08249,USA,Basic,u18,0.0,955.8,0



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [19]:
%%bigquery --project boxwood-veld-471119-r6
CREATE OR REPLACE MODEL `boxwood-veld-471119-r6.netflix.churn_model`
OPTIONS(model_type='logistic_reg', input_label_cols=['churn_label']) AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `boxwood-veld-471119-r6.netflix.churn_features`;

Query is running:   0%|          |

""



## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [20]:
%%bigquery --project boxwood-veld-471119-r6
SELECT * FROM ML.EVALUATE(MODEL `boxwood-veld-471119-r6.netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.0,0.0,0.86357,0.0,0.399021,0.462921


In [21]:
%%bigquery --project boxwood-veld-471119-r6
SELECT * FROM ML.CONFUSION_MATRIX(MODEL `boxwood-veld-471119-r6.netflix.churn_model`);

Query is running:   0%|          |

Downloading:   0%|          |

,expected_label,_0,_1
0,0,1766,0
1,1,279,0



## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [23]:
%%bigquery --project boxwood-veld-471119-r6
SELECT
    user_id,
    predicted_churn_label,
    (SELECT prob FROM UNNEST(predicted_churn_label_probs) WHERE label = 1) AS churn_probability
FROM ML.PREDICT(MODEL `boxwood-veld-471119-r6.netflix.churn_model`,
      (SELECT user_id, region, plan_tier, total_minutes, avg_rating
       FROM `boxwood-veld-471119-r6.netflix.churn_features`));

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,predicted_churn_label,churn_probability
0,user_02716,0,0.146578
1,user_05854,0,0.149085
2,user_03784,0,0.144874
3,user_05931,0,0.148010
4,user_07856,0,0.149218
...,...,...,...
10295,user_03189,0,0.146098
10296,user_04924,0,0.146178
10297,user_08385,0,0.143654
10298,user_09019,0,0.144138
